In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf

%load_ext tensorboard
import datetime

from tensorflow.keras.utils import to_categorical

# ignore Deprecation Warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense      # create layers

df = pd.read_csv('/content/titanic.csv')

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)
df = df_train.append(df_test , ignore_index = True)

# some quick inspections
df_train.shape, df_test.shape, df_train.columns.values

df['Pclass'].isnull().sum(axis=0)

# inspect the correlation between Pclass and Survived
df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

df['Title'] = df.Name.map( lambda x: x.split(',')[1].split( '.' )[0].strip())

# inspect the amount of people for each title
df['Title'].value_counts()

df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace(['Mme','Lady','Ms'], 'Mrs')
df.Title.loc[ (df.Title !=  'Master') & (df.Title !=  'Mr') & (df.Title !=  'Miss') 
             & (df.Title !=  'Mrs')] = 'Others'

# inspect the correlation between Title and Survived
df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

# inspect the amount of people for each title
df['Title'].value_counts()

df = pd.concat([df, pd.get_dummies(df['Title'])], axis=1).drop(labels=['Name'], axis=1)

# check if there is any NAN
df.Sex.isnull().sum(axis=0)

# inspect the correlation between Sex and Survived
df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

# map the two genders to 0 and 1
df.Sex = df.Sex.map({'male':0, 'female':1})

# check if there is any NAN
df.Age.isnull().sum(axis=0)

# check if there is any NAN
df.SibSp.isnull().sum(axis=0), df.Parch.isnull().sum(axis=0)

# create a new feature "Family"
df['Family'] = df['SibSp'] + df['Parch'] + 1

# inspect the correlation between Family and Survived
df[['Family', 'Survived']].groupby(['Family'], as_index=False).mean()

# inspect the amount of people for each Family size
df['Family'].value_counts()

df.Family = df.Family.map(lambda x: 0 if x > 4 else x)
df[['Family', 'Survived']].groupby(['Family'], as_index=False).mean()

df['Family'].value_counts()

# check if there is any NAN
df.Ticket.isnull().sum(axis=0)

df.Ticket = df.Ticket.map(lambda x: x[0])

# inspect the correlation between Ticket and Survived
df[['Ticket', 'Survived']].groupby(['Ticket'], as_index=False).mean()

df[['Ticket', 'Fare']].groupby(['Ticket'], as_index=False).mean()

df[['Ticket', 'Pclass']].groupby(['Ticket'], as_index=False).mean()

# check if there is any NAN
df.Fare.isnull().sum(axis=0)

df.Ticket[df.Fare.isnull()]

df.Pclass[df.Fare.isnull()]

df.Cabin[df.Fare.isnull()]

df.Embarked[df.Fare.isnull()]

guess_Fare = df.Fare.loc[ (df.Ticket == '3') & (df.Pclass == 3) & (df.Embarked == 'S')].median()
df.Fare.fillna(guess_Fare , inplace=True)

# bin Fare into five intervals with equal amount of people
df['Fare-bin'] = pd.qcut(df.Fare,5,labels=[1,2,3,4,5]).astype(int)

# inspect the correlation between Fare-bin and Survived
df[['Fare-bin', 'Survived']].groupby(['Fare-bin'], as_index=False).mean()

# check if there is any NAN
df.Cabin.isnull().sum(axis=0)

df = df.drop(labels=['Cabin'], axis=1)

# check if there is any NAN
df.Embarked.isnull().sum(axis=0)

# fill the NAN
df.Embarked.fillna('S' , inplace=True )

df = df.drop(labels='Embarked', axis=1)

# notice that instead of using Title, we should use its corresponding dummy variables 
df_sub = df[['Age','Master','Miss','Mr','Mrs','Others','Fare-bin','SibSp']]

X_train  = df_sub.dropna().drop('Age', axis=1)
y_train  = df['Age'].dropna()
X_test = df_sub.loc[np.isnan(df.Age)].drop('Age', axis=1)

regressor = RandomForestRegressor(n_estimators = 300)
regressor.fit(X_train, y_train)
y_pred = np.round(regressor.predict(X_test),1)
df.Age.loc[df.Age.isnull()] = y_pred

df.Age.isnull().sum(axis=0) # no more NAN now

bins = [ 0, 4, 12, 18, 30, 50, 65, 100] # This is somewhat arbitrary...
age_index = (1,2,3,4,5,6,7)
#('baby','child','teenager','young','mid-age','over-50','senior')
df['Age-bin'] = pd.cut(df.Age, bins, labels=age_index).astype(int)

df[['Age-bin', 'Survived']].groupby(['Age-bin'],as_index=False).mean()

df['Ticket'] = df['Ticket'].replace(['A','W','F','L','5','6','7','8','9'], '4')

# dummy encoding
df = pd.get_dummies(df,columns=['Ticket'])

df = df.drop(labels=['SibSp','Parch','Age','Fare','Title'], axis=1)
y_train = df[0:712]['Survived'].values
X_train = df[0:712].drop(['Survived','PassengerId'], axis=1).values
X_test  = df[712:].drop(['Survived','PassengerId'], axis=1).values
y_test =  df[712:]['Survived'].values

print(X_train.shape)
print(X_test.shape)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


(712, 17)
(179, 17)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [6]:
initializer0 = keras.initializers.RandomUniform(minval = -2, maxval =-1)
initializer1 = keras.initializers.RandomUniform(minval = 1, maxval =2)
initializer2 = keras.initializers.RandomUniform(minval = 0, maxval =0.005)

class Diffact(keras.layers.Layer):
    def __init__(self):
        super(Diffact, self).__init__()
        self.k0 = self.add_weight(name='k0', shape = (), initializer=initializer0, trainable=True)
        self.k1 = self.add_weight(name='k1', shape = (), initializer=initializer1, trainable=True)
        self.k2 = self.add_weight(name='k2', shape = (), initializer=initializer2, trainable=True)
        
    def call(self, inputs):
        return self.k0 + tf.multiply(inputs, self.k1) + tf.multiply(tf.multiply(inputs,inputs), self.k2)

from keras import backend as K
class Mish(keras.layers.Layer):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X_input = Input(input_shape)
        >>> X = Mish()(X_input)
    '''

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        base_config = super(Mish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

In [7]:
for i in range(1):  
  # Initialising the NN
  model = Sequential()
  diffact=Diffact()

  # layers
  model.add(Dense(units = 9, kernel_initializer = 'uniform',input_dim = 17))
  model.add(diffact)
  model.add(Dense(units = 9, kernel_initializer = 'uniform'))
  model.add(diffact)
  model.add(Dense(units = 5, kernel_initializer = 'uniform'))
  model.add(diffact)
  model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

  # Compiling the ANN
  model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True), loss = 'binary_crossentropy', metrics = ['accuracy'])
  model.summary()

  log_dir = "logs/fit4/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


  # Train the ANN
  model.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split=0.33, callbacks=[tensorboard_callback], verbose=1)

  score = model.evaluate(X_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  print("AF coefficients (weights) {}".format(diffact.get_weights()))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 9)                 162       
_________________________________________________________________
diffact_1 (Diffact)          multiple                  3         
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 50        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 6         
Total params: 311
Trainable params: 311
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
15/15 [==============================] - 1s 23ms/step - loss: 0.6845 - accuracy: 0.5963 - val_loss: 0.6737 - v

In [8]:
%tensorboard --logdir logs/fit4 --port=8042

Reusing TensorBoard on port 8042 (pid 708), started 0:29:09 ago. (Use '!kill 708' to kill it.)

<IPython.core.display.Javascript object>